In [94]:
import cv2
import torch
import numpy as np
import time


def calculate_green_signal(vehicle_count):
    """
    Calculate the green signal duration based on vehicle count.
    Example: 2 seconds per vehicle with a minimum of 5 seconds.
    """
    return max(5, vehicle_count * 2)


# Load YOLOv5 model from PyTorch Hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Initialize video capture (use your video file or webcam)
cap = cv2.VideoCapture('cars.avi')  # Replace with your video file path

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Loop over video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or cannot read the frame.")
        break

    # Convert the frame to RGB for YOLOv5
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform detection
    results = model(img)

    # Get the results as a pandas dataframe
    results_data = results.pandas().xywh[0]

    # Filter detections by object class (cars, trucks, etc.)
    car_class_id = 2  # Adjust based on YOLO class ID for cars
    vehicles = results_data[results_data['class'] == car_class_id]

    # Count detected vehicles
    vehicle_count = len(vehicles)

    # Calculate green signal duration
    green_signal_duration = calculate_green_signal(vehicle_count)

    # Draw bounding boxes and display vehicle information
    for index, row in vehicles.iterrows():
        x1 = int(row['xcenter'] - row['width'] / 2)
        y1 = int(row['ycenter'] - row['height'] / 2)
        x2 = int(row['xcenter'] + row['width'] / 2)
        y2 = int(row['ycenter'] + row['height'] / 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # Display vehicle count and green signal duration on the frame
    cv2.putText(frame, f'Vehicle Count: {vehicle_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Green Signal: {green_signal_duration} sec', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the frame with bounding boxes and text
    cv2.imshow('Traffic Signal Control', frame)

    # Wait for the appropriate green signal duration (in seconds)
    time.sleep(green_signal_duration)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\suppa/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  v7.0-392-gf003c3df Python-3.12.0 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users/suppa/yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [99]:
import cv2
print(cv2.getBuildInformation())




General configuration for OpenCV 4.10.0 =====================================
  Version control:               4.10.0

  Platform:
    Timestamp:                   2024-06-17T17:58:35Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.24.2
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (8 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (36 files):           + SSSE3 SSE4_